In [1]:
import hickle
from ase.io import read
import pyscf.pbc.gto as pbcgto
import pyscf.pbc.tools.pyscf_ase as pyscf_ase
from pyscf.pbc.tools.k2gamma import get_phase, kpts_to_kmesh, k2gamma
import numpy as np
import pyscf.pbc.scf as pbcscf
from ase.visualize import view

/home/pegolo/micromamba/envs/sci/lib/python3.11/site-packages/pyscf/dft/libxc.py:771: UserWarning: Since PySCF-2.3, B3LYP (and B3P86) are changed to the VWN-RPA variant, corresponding to the original definition by Stephens et al. (issue 1480) and the same as the B3LYP functional in Gaussian. To restore the VWN5 definition, you can put the setting "B3LYP_WITH_VWN5 = True" in pyscf_conf.py
  warnings.warn('Since PySCF-2.3, B3LYP (and B3P86) are changed to the VWN-RPA variant, '


In [10]:
def print_matrix(matrix):
    for row in matrix:
        for v in row:
            print(f'{v:>12.6f}', end = ' ')
        print()

In [95]:
from ase.atoms import Atoms
frame= Atoms('He', positions = [[0,0,0]], cell=[10,10,1])
frame.center()
frame22 = frame.repeat([2,2,1])
#view([frame, frame22])

In [97]:
# view([frame, frame22])

In [16]:
# frame = read('examples/data/periodic/graphene_supercell_variety/graphene_variety.xyz', '1')


In [79]:
kmesh = [8,8,1]
cell = pbcgto.Cell()
cell.atom = pyscf_ase.ase_atoms_to_pyscf(frame)

cell.basis = 'def2-svp'
cell.a = frame.cell
cell.verbose = 4
print('-------------------------------------------------------------------------------', flush= True)
cell.build()

kmesh = kmesh
print('KMESH', kmesh)
kpts = cell.make_kpts(kmesh)
#kmesh = [np.int32(np.max([1, np.ceil(2*np.pi*np.linalg.norm(vec)/kspacing)])) for vec in frame.cell.reciprocal().array]

nao = cell.nao
kmf = pbcscf.KRHF(cell, kpts = cell.make_kpts(kmesh))

kmf = kmf.density_fit()
kmf.init_guess_by_minao() #('minao')
kmf.max_cycle = 2000
kmf.conv_tol = 1e-8 
kmf.conv_tol_grad = 1e-8

e1= kmf.kernel()

-------------------------------------------------------------------------------
System: uname_result(system='Linux', node='cosmopc3', release='5.15.0-91-generic', version='#101-Ubuntu SMP Tue Nov 14 13:30:08 UTC 2023', machine='x86_64')  Threads 24
Python 3.11.7 | packaged by conda-forge | (main, Dec 23 2023, 14:43:09) [GCC 12.3.0]
numpy 1.26.3  scipy 1.10.1
Date: Fri Mar 15 13:59:38 2024
PySCF version 2.4.0
PySCF path  /home/pegolo/micromamba/envs/sci/lib/python3.11/site-packages/pyscf

[CONFIG] conf_file None
[INPUT] verbose = 4
[INPUT] num. atoms = 1
[INPUT] num. electrons = 2
[INPUT] charge = 0
[INPUT] spin (= nelec alpha-beta = 2S) = 0
[INPUT] symmetry False subgroup None
[INPUT] Mole.unit = angstrom
[INPUT] Symbol           X                Y                Z      unit          X                Y                Z       unit  Magmom
[INPUT]  1 He     1.000000000000   1.000000000000   0.500000000000 AA    1.889726124565   1.889726124565   0.944863062283 Bohr   0.0

nuclear repulsio

In [80]:
supercell = pbcgto.Cell()
supercell.atom = pyscf_ase.ase_atoms_to_pyscf(frame22)

supercell.basis = 'def2-svp'
supercell.a = frame22.cell
supercell.verbose = 4
print('-------------------------------------------------------------------------------', flush= True)
supercell.build()

kmesh = [4,4,1] #kmesh
print('KMESH', kmesh)
kpts = supercell.make_kpts(kmesh)
#kmesh = [np.int32(np.max([1, np.ceil(2*np.pi*np.linalg.norm(vec)/kspacing)])) for vec in frame.cell.reciprocal().array]

nao = cell.nao
superkmf = pbcscf.KRHF(supercell, kpts = kpts)

superkmf = superkmf.density_fit()
superkmf.max_cycle = 500
superkmf.conv_tol = 1e-8 
superkmf.conv_tol_grad = 1e-8

e2 = superkmf.kernel()

-------------------------------------------------------------------------------
System: uname_result(system='Linux', node='cosmopc3', release='5.15.0-91-generic', version='#101-Ubuntu SMP Tue Nov 14 13:30:08 UTC 2023', machine='x86_64')  Threads 24
Python 3.11.7 | packaged by conda-forge | (main, Dec 23 2023, 14:43:09) [GCC 12.3.0]
numpy 1.26.3  scipy 1.10.1
Date: Fri Mar 15 14:01:06 2024
PySCF version 2.4.0
PySCF path  /home/pegolo/micromamba/envs/sci/lib/python3.11/site-packages/pyscf

[CONFIG] conf_file None
[INPUT] verbose = 4
[INPUT] num. atoms = 4
[INPUT] num. electrons = 8
[INPUT] charge = 0
[INPUT] spin (= nelec alpha-beta = 2S) = 0
[INPUT] symmetry False subgroup None
[INPUT] Mole.unit = angstrom
[INPUT] Symbol           X                Y                Z      unit          X                Y                Z       unit  Magmom
[INPUT]  1 He     1.000000000000   1.000000000000   0.500000000000 AA    1.889726124565   1.889726124565   0.944863062283 Bohr   0.0
[INPUT]  2 He    

In [81]:
e2/e1

4.000000000366593

In [82]:
fock = kmf.get_fock()
over = kmf.get_ovlp()
sfock = superkmf.get_fock()
sover = superkmf.get_ovlp()



******** <class 'pyscf.pbc.df.rsdf_builder._RSNucBuilder'> ********
mesh = [7 7 5] (245 PWs)
ke_cutoff = 12.436968235142707
omega = 0.45761683343257953
exclude_d_aux = False
exclude_dd_block = True
j2c_eig_always = False
has_long_range = True


******** <class 'pyscf.pbc.df.rsdf_builder._RSNucBuilder'> ********
mesh = [13 13  5] (845 PWs)
ke_cutoff = 12.436968235142707
omega = 0.45761683343257953
exclude_d_aux = False
exclude_dd_block = True
j2c_eig_always = False
has_long_range = True


Very important: $\frac{123.5199}{30.34544157907528} \simeq \frac{8}{2}$

In [83]:
print_matrix(fock[0][:5,:5].real)

   -0.926783    -2.275759     0.000000     0.000000     0.000000 
   -2.275759    -2.075280     0.000000     0.000000    -0.000000 
    0.000000     0.000000     2.315694    -0.000000     0.000000 
    0.000000     0.000000    -0.000000     2.315694    -0.000000 
    0.000000    -0.000000     0.000000    -0.000000     0.687524 


In [84]:
print_matrix(sfock[0][:5,:5].real)

   -0.879621    -1.803890    -0.000000    -0.000000     0.000000 
   -1.803890    -1.159514     0.000000     0.000000     0.000000 
   -0.000000     0.000000     2.192807     0.000000     0.000000 
   -0.000000     0.000000     0.000000     2.192807    -0.000000 
    0.000000     0.000000     0.000000    -0.000000     0.687724 


In [85]:
print_matrix(over[0][:5,:5].real)

    1.152868     1.355004    -0.000000    -0.000000    -0.000000 
    1.355004     3.733936    -0.000000    -0.000000    -0.000000 
    0.000000     0.000000     1.310968    -0.000000     0.000000 
   -0.000000     0.000000    -0.000000     1.310968    -0.000000 
    0.000000     0.000000     0.000000    -0.000000     0.116970 


In [86]:
print_matrix(sover[0][:5,:5].real)

    1.152430     1.202417     0.000000    -0.000000    -0.000000 
    1.202417     2.433568     0.000000     0.000000    -0.000000 
    0.000000     0.000000     1.336998     0.000000    -0.000000 
   -0.000000    -0.000000     0.000000     1.336998    -0.000000 
    0.000000     0.000000    -0.000000    -0.000000     0.116601 


In [89]:
from ase.visualize import view 
from ase.io import read
fr = read('examples/data/periodic/c2/C2_174.extxyz', ':')
fr2 = read('examples/data/periodic/c4/C4_174.extxyz', ':')


In [92]:
print(fr[0].positions)
print()
print(fr[0].cell)

[[-0.08351854 -0.00307582  1.62727708]
 [-0.05879422  1.33906047  1.60493484]]

Cell([[2.321189, 0.0, 0.0], [-1.155982, 1.986034, 0.0], [-0.37381, -0.001243, 106.457295]])


In [93]:
print(fr2[0].positions)
print()
print(fr2[0].cell)

[[-0.0785514  -0.00445846  1.63375397]
 [-0.04356392  1.33671894  1.60259347]
 [ 2.25346259  0.00459853  1.62258234]
 [ 2.26782038  1.33442629  1.60027754]]

Cell([[4.642378, 0.0, 0.0], [-1.155982, 1.986034, 0.0], [-0.37381, -0.001243, 106.457295]])


In [94]:
view(fr[0])

<Popen: returncode: None args: ['/home/pegolo/micromamba/envs/sci/bin/python...>